In [1]:
import os
import pandas as pd
import zipfile
import pyodbc
import requests
import io
from datetime import datetime, timedelta
import sqlalchemy
import urllib
import warnings
warnings.filterwarnings('ignore')

In [2]:
NEMO = "DTE_EMISION"

URL = f"https://api.cammesa.com/pub-svc/public/"

method_id = "findDocumentosByNemoRango?" #ID
method_zip = "findAllAttachmentZipByNemoId?"

fecha_desde = "2024-03-31T00:00:00.000-03:00"
fecha_hasta = "2024-04-30T23:59:59.000-03:00"

In [3]:
url_doc_id = f"{URL}{method_id}fechadesde={fecha_desde}&fechahasta={fecha_hasta}&nemo={NEMO}"

In [5]:
try:
    with requests.get(url_doc_id) as response:
        if response.status_code == 200:
            PPO=response.json()
            doc_id = PPO[-1]['id']
        else:
            print("La solicitud falló con el código de estado:", response.status_code)
except requests.exceptions.RequestException as e:
    # Manejar la excepción
    print("Error al realizar la solicitud:", e)

In [6]:
url_zip = f"{URL}{method_zip}docId={doc_id}&nemo={NEMO}"

In [16]:
try:
    with requests.get(url_zip) as r:

        if r.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(r.content)) as primer_zip:
                
                files_inside = primer_zip.namelist()
                DTE_name = primer_zip.namelist()[0]
                DTE_fecha = primer_zip.namelist()[0][:7]
                
                path_dte = f"../.zips/{DTE_fecha}"   
                
                #primer_zip.extractall(path=f"../.zips/{DTE_fecha}") #Extraer todos los archivos del primer .zip
                primer_zip.extract(f"{DTE_name}", path= path_dte)

                with zipfile.ZipFile(f"{path_dte}/{DTE_name}", 'r') as segundo_zip:

                    DTE_mdb_name = f"{DTE_fecha}.MDB"

                    segundo_zip.extract(f"{DTE_mdb_name}", path=path_dte)
                
        else:
            print("La solicitud falló con el código de estado:", response.status_code)

except requests.exceptions.RequestException as e:
    # Manejar la excepción
    print("Error al realizar la solicitud:", e)